# Embedding

Nesse notebook consta o processo de criar um ebedding do dhbb.

Perceba que no diretório apontado para salvar o índice, serão criados dois arquivos: um índice .faiss (base vetorial) e um arquivo .pkl (metadados das chunks)

In [3]:
import os
import glob
import yaml

from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.documents import Document
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS

### Configurações de embedding e diretórios

In [10]:
INPUT_DIR = "/disco/dhbb/text" # Diretório com os verbetes
INDEX_SAVE_PATH = "/disco/indexes" # Diretório para salvar a base vetorial
EMBEDDING_MODEL = "sentence-transformers/all-MiniLM-L6-v2" # Modelo de embedding

## 1. Carregar arquivos .text

In [11]:
text_files = glob.glob(os.path.join(INPUT_DIR, "**/*.text"), recursive=True)

if not text_files:
    raise ValueError(f"Nenhum arquivo .text encontrado em {INPUT_DIR}")

Repare que o tamanho dos verbetes costuma ser bem inconsistente. É importante levar isso em consideração quando definimos os parâmetros para criação de Documents e chunks.

## 2. Processar arquivos e criar objetos Document

Utiliza-se os cabeçalhos YAML dos verbetes para gerar os metadados.

In [ ]:
documents = []
for file_path in text_files:
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            content = f.read()

            # Dividir o cabeçalho YAML do corpo do texto
            if content.startswith("---"):
                parts = content.split("---", 2)
                if len(parts) >= 3:
                    _, header, body = parts
                    metadata = yaml.safe_load(header)
                else:
                    body = content
                    metadata = {}
            else:
                body = content
                metadata = {}
            
            metadata["source"] = os.path.basename(file_path)
            metadata["file_path"] = file_path

            
            # Criar objeto Document
            documents.append(Document(
                page_content=content,
                metadata=metadata
            ))
    except Exception as e:
        print(f"Erro ao processar {file_path}: {str(e)}")

## 3. Dividir documentos em chunks
É importante definir bem os parâmetros de chunk size e chunk overlap, pois estes tem forte impacto na criação do embedding:

- chunk_size: Uma vez que cada verbete foi transformado num Document, cada Document é quebrado em chunks. Se $chunk\_size = n$, então cada chunk terá no máximo $n$ caracteres.
- chunk_overlap: Para evitar que a informação de uma chunk seja completamente descontextualizada, se $chunk\_overlap=m$, os $m$ últimos caracteres da $i$-ésima chunk serão repetidos no início da $(i+1)$-ésima chunk.


In [14]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=7000,
    chunk_overlap=500
)

split_docs = text_splitter.split_documents(documents)  # Método correto para documentos

## 4. Criar embeddings e índice

In [16]:

embeddings = HuggingFaceEmbeddings(model_name=EMBEDDING_MODEL)

vectorstore = FAISS.from_documents(
    documents=split_docs,  # Já são objetos Document
    embedding=embeddings
)

os.makedirs(os.path.dirname(INDEX_SAVE_PATH), exist_ok=True)
vectorstore.save_local(INDEX_SAVE_PATH)
print(f"Índice criado com {len(split_docs)} chunks em {INDEX_SAVE_PATH}")

Índice criado com 12756 chunks em /disco/indexes
